In [60]:
import os
import pandas as pd

#### Players

In [61]:
# Read in data
players = {}
path = '../data/data_raw/players'
for file in os.listdir(path):
    if file.split('.')[1]=='csv':
        players[file.split('.')[0]] = pd.read_csv(os.path.join(path,file))

for key, value in players.items():
    print(value.head())

   G#      Date   Tm home/away  Opp   Result Pos  FantPt
0   1   9/12/10  PHI       NaN  GNB  L 20-27  RB    14.2
1   2   9/19/10  PHI         @  DET  W 35-32  RB    30.8
2   3   9/26/10  PHI         @  JAX   W 28-3  RB     6.1
3   4   10/3/10  PHI       NaN  WAS  L 12-17  RB    15.4
4   5  10/10/10  PHI         @  SFO  W 27-24  RB    19.8
   G#      Date   Tm home/away  Opp   Result Pos  FantPt
0   1   9/12/10  NWE       NaN  CIN  W 38-24  QB   22.32
1   2   9/19/10  NWE         @  NYJ  L 14-28  QB   11.92
2   3   9/26/10  NWE       NaN  BUF  W 38-30  QB   22.68
3   4   10/4/10  NWE         @  MIA  W 41-14  QB   10.72
4   5  10/17/10  NWE       NaN  BAL  W 23-20  QB   11.78


In [62]:
# define data types; drop columns we don't need; rename columns for ease of uses
unneeded_cols = [
    'G#', 
    'Tm', 
    'Result', 
    'Pos'
]
renamed_cols = {'home/away':'home'} # Convert this to boolean

for key, value in players.items():
    value.drop(unneeded_cols, axis=1, inplace=True)
    value.rename(columns=renamed_cols, inplace=True)
    value.columns = [col.lower() for col in value.columns]
    value['date'] = pd.to_datetime(value['date'], errors='coerce')
    value['home'] = [1 if pd.isnull(loc) else 0 for loc in value['home']]
    
for key, value in players.items():
    print(value.head())

        date  home  opp  fantpt
0 2010-09-12     1  GNB    14.2
1 2010-09-19     0  DET    30.8
2 2010-09-26     0  JAX     6.1
3 2010-10-03     1  WAS    15.4
4 2010-10-10     0  SFO    19.8
        date  home  opp  fantpt
0 2010-09-12     1  CIN   22.32
1 2010-09-19     0  NYJ   11.92
2 2010-09-26     1  BUF   22.68
3 2010-10-04     0  MIA   10.72
4 2010-10-17     1  BAL   11.78


#### Games

In [64]:
games = pd.read_csv('../data/data_raw/games/game_metadata.csv')
games.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/02/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,09/03/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,09/04/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,09/09/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,09/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


We can't directly join these dataframes because 'games' uses full team name while the 'players' dfs use abbreviation. Since we only have two teams, we can just filter on team name and join by date. To scale this in the future, we would want to build out this process so we can directly join.

In [65]:
games['schedule_date'] = pd.to_datetime(games['schedule_date'], errors='coerce')

games_patriots = games.loc[(games['team_home']=='New England Patriots') | (games['team_away']=='New England Patriots')]
games_eagles = games.loc[(games['team_home']=='Philadelphia Eagles') | (games['team_away']=='Philadelphia Eagles')]

print(games_patriots.shape)
print(games_eagles.shape)

(843, 17)
(869, 17)


In [66]:
for key, value in players.items():
    if key =='brady':
        players[key] = value.merge(games_patriots, how='inner', left_on='date', right_on='schedule_date')
    else:
        players[key] = value.merge(games_eagles, how='inner', left_on='date',  right_on='schedule_date')

In [68]:
players['brady'].columns

Index([u'date', u'home', u'opp', u'fantpt', u'schedule_date',
       u'schedule_season', u'schedule_week', u'schedule_playoff', u'team_home',
       u'score_home', u'score_away', u'team_away', u'team_favorite_id',
       u'spread_favorite', u'over_under_line', u'stadium', u'stadium_neutral',
       u'weather_temperature', u'weather_wind_mph', u'weather_humidity',
       u'weather_detail'],
      dtype='object')

In [70]:
# More column manipulation
unneeded_cols = [
    'schedule_date', 
    'schedule_season', 
    'schedule_week', 
    'schedule_playoff',
    'team_home',
    'score_home',
    'team_away',
    'score_away',
    'stadium',
    'stadium_neutral'
]
renamed_cols = {'team_favorite_id':'favorite'}

for key, value in players.items():
    value.drop(unneeded_cols, axis=1, inplace=True)
    value.rename(columns=renamed_cols, inplace=True)
    if key=='brady':
        team_id = 'NE'
    else:
        team_id = 'PHI'
    value['favorite'] = [1 if code==team_id else 0 for code in value['favorite']]
    
for key, value in players.items():
    print(value.head())

        date  home  opp  fantpt  favorite  spread_favorite over_under_line  \
0 2010-09-12     1  GNB    14.2         0             -3.0              48   
1 2010-09-19     0  DET    30.8         1             -6.5              41   
2 2010-09-26     0  JAX     6.1         1             -3.0              45   
3 2010-10-03     1  WAS    15.4         1             -5.0            43.5   
4 2010-10-10     0  SFO    19.8         0             -3.0            38.5   

   weather_temperature  weather_wind_mph weather_humidity weather_detail  
0                 69.0               4.0               72            NaN  
1                 72.0               0.0              NaN           DOME  
2                 90.0               5.0               67            NaN  
3                 63.0               1.0               57            NaN  
4                 75.0               1.0               51            NaN  
        date  home  opp  fantpt  favorite  spread_favorite over_under_line  \
0 2

In [94]:
# Save data to new folder so we can use it easily
for key, value in players.items():
    value.to_csv('../data/data_modified/players/{}.csv'.format(key), index=False)